In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas_profiling import ProfileReport

In [6]:
df = pd.read_csv('../data/census.csv')

In [7]:
df.head(1)

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K


In [8]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [9]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]